##This is programming lab 2, healthcare analytics
##---Downloading the Medicare Hospital Compare Data and Loading it into SQL

In [ ]:
import requests, os, zipfile,openpyxl, csv, sqlite3, re
from pathlib import Path

staging_dir_nm = "staging"
os.mkdir(staging_dir_nm)

url="https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"

r = requests.get(url)

zip_file_nm = os.path.join(staging_dir_nm, "Hospital_Revised_Flatfiles.zip")
zf = open(zip_file_nm, "wb")

zf.write(r.content)

zf.close()

z = zipfile.ZipFile(zip_file_nm,"r")
z.extractall(staging_dir_nm)
z.close()

def _get_col_datatypes(fin):
    dr = csv.DictReader(fin) # comma is default delimiter
    fieldTypes = {}
    for entry in dr:
        feildslLeft = [f for f in dr.fieldnames if f not in fieldTypes.keys()]
        if not feildslLeft: break # We're done
        for field in feildslLeft:
            fieldTypes[field] = "TEXT"
    return fieldTypes


def escapingGenerator(f):
    for line in f:
        yield line.encode("ascii", "xmlcharrefreplace").decode("cp1252")      

def parsingNameByLogic(toBeTransformed, nm_type):
    Transformed = re.sub(r"\-", "_", toBeTransformed.lower())
    Transformed = re.sub(r"\s", "_", Transformed)
    Transformed = re.sub(r"\-", "_", Transformed)
    Transformed = re.sub(r"\%", "pct", Transformed)
    Transformed = re.sub(r"\/", "_", Transformed)
    
    if nm_type == 'TABLE':
        if re.match(r"^[^a-z]", Transformed):
            Transformed = "t_" + str(Transformed)
    elif nm_type == 'COLUMN':
        if re.match(r"^[^a-z]", Transformed):
            Transformed = "c_" + str(Transformed)
            
    return Transformed
    
        
def csvToDb(csvFile, outputToFile = False):
    
    #Parsing CSV File name (Table Name)
    tbl_nm = re.sub(r"\.csv", "", csvFile)
    tbl_nm = re.sub(r"^[s][t][a][g][i][n][g][\\]","", tbl_nm)
    tbl_nm = parsingNameByLogic(tbl_nm,'TABLE')
       
    
    #with open(csvFile,mode='r', encoding="ISO-8859-1") as fin:
    with open(csvFile,mode='r', encoding="cp1252") as fin:
        dt = _get_col_datatypes(fin)

        fin.seek(0)

        reader = csv.DictReader(fin)

        # Keep the order of the columns name just as in the CSV
        fields = reader.fieldnames
        cols = []

        # Set field and type
        for f in fields:
            cols.append("%s %s" % (parsingNameByLogic(f, 'COLUMN'), dt[f]))

        
        # Generate create table statement:
        stmt = "CREATE TABLE " + tbl_nm +" (%s)" % ",".join(cols)

        con = sqlite3.connect("medicare_hospital_compare.db")
        #con = sqlite3.connect(":memory:")
        cur = con.cursor()
        cur.execute('DROP TABLE {tn} '.format(tn=tbl_nm))
        con.commit()
        
        cur.execute(stmt)
        con.commit()

        fin.seek(0)


        reader = csv.reader(escapingGenerator(fin))

        # Generate insert statement:
        stmt = "INSERT INTO " + tbl_nm + " VALUES(%s);" % ','.join('?' * len(cols))
        
        cur.executemany(stmt, reader)
        con.commit()

    return con

directory_in_str = "staging"

pathlist = Path(directory_in_str).glob('**/*.csv')
for path in pathlist:
    # because path is object not string
    path_in_str = str(path)
    
    if (path_in_str != "staging\\FY2015_Percent_Change_in_Medicare_Payments.csv")\   #This is bad file
         and (path_in_str != "staging\\MORT_READM_April2017.csv")\   #Can't figure out how to import this
         and (path_in_str != "staging\\PSI_April2017.csv"):          #Can't figure out how to import this
        csvToDb(path_in_str)

##Download MS Excel Workbook of In House Proprietary Hospital Rankings and Focus List of States

In [2]:
import requests,openpyxl,sqlite3

"""
url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"

r = requests.get(url)
type(r.content)
f = open("hospital_ranking_focus_states.xlsx","wb")
f.write(r.content)
f.close()
"""

wb = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")

for sheet_name in wb.get_sheet_names():
    print(sheet_name)
    
sheet_1 = wb.get_sheet_by_name("Hospital National Ranking")
sheet_2 = wb.get_sheet_by_name("Focus States")

con = sqlite3.connect("medicare_hospital_compare.db")
cur = con.cursor()

# Generate create table statement:
"""
stmt = "CREATE TABLE hospital_national_ranking (provider_id, ranking)"
cur.execute(stmt)

stmt = "CREATE TABLE focus_states (state_name, state_abbreviation)"
cur.execute(stmt)

con.commit()
"""

# Create the 1st sheet INSERT INTO sql query
query = " INSERT INTO hospital_national_ranking (provider_id, ranking) VALUES ("

# Create a For loop to iterate through each row in the XLS file, starting at row 2 to skip the headers
i = 2
while sheet_1.cell(row = i, column = 1).value != None:
    col1 = str(sheet_1.cell(row = i, column = 1).value)
    col2 = str(sheet_1.cell(row = i, column = 2).value)
    i += 1

    # Assign values from each row
    test = query + "\"" +col1 + "\" , \""+ col2 + "\")"

    # Execute sql Query
    #cur.execute(test)

# Close the cursor
cur.close()
# Commit the transaction
con.commit()

# Create the 2nd  sheet INSERT INTO sql query
query = " INSERT INTO focus_states (state_name, state_abbreviation) VALUES ("

# Create a For loop to iterate through each row in the XLS file, starting at row 2 to skip the headers
i = 2
while sheet_2.cell(row = i, column = 1).value != None:
    col1 = str(sheet_2.cell(row = i, column = 1).value)
    col2 = str(sheet_2.cell(row = i, column = 2).value)
    i += 1

    # Assign values from each row
    test = query + "\"" +col1 + "\" , \""+ col2 + "\")"
    #print (test)
    # Execute sql Query
    #cur.execute(test)
    
# Close the cursor
cur.close()
# Commit the transaction
con.commit()


# Close the database connection
con.close()

Hospital National Ranking
Focus States


##Create the Hospital Ranking MS Excel Workbook

##Create the Measures Statistical Analysis MS Excel Workbook

In [2]:
import sqlite3,re,math
from xlsxwriter.workbook import Workbook

class StdevFunc:
    def __init__(self):
        self.M = 0.0
        self.S = 0.0
        self.k = 1

    def step(self, value):
        if value is None:
            return
        tM = self.M
        self.M += (value - tM) / self.k
        self.S += (value - tM) * (value - self.M)
        self.k += 1

    def finalize(self):
        if self.k < 3:
            return None
        return math.sqrt(self.S / (self.k-2))


sqlite_file = 'medicare_hospital_compare.db'    # name of the sqlite database file

tbl_hospital_info = 'hospital_general_information'
tbl_hospital_ranking = 'hospital_national_ranking'
tbl_focus_state = 'focus_states'
tbl_time_effect = 'timely_and_effective_care___hospital'

tbl_time_effect_temp = 'time_effect_temp'      #This one is for my design

column_1 = 'ranking'
column_state_abb = ''   #state abbreviation
column_state_full = ''  #state fullname

# Connecting to the database file

conn = sqlite3.connect(sqlite_file)

conn.create_aggregate("stdev", 1, StdevFunc)

c = conn.cursor()

############################Create the Hospital Ranking MS Excel Workbook###########################################

##Nationwide Sheet
c.execute('SELECT t1.provider_id, t2.hospital_name, t2.city, t2.state, t2.county_name, t1.ranking FROM {tn_1} as t1 JOIN {tn_2} as t2 ON t1.provider_id = t2.provider_id WHERE CAST({rank} AS INTEGER) between 1 and 100'.\
                            
            format(tn_1=tbl_hospital_ranking, tn_2=tbl_hospital_info,rank= column_1))

all_rows = c.fetchall()

file_name="hospital_ranking.xlsx"

workbook = Workbook(file_name)
sheet_1 = workbook.add_worksheet("Nationwide")

for r, row in enumerate(all_rows):
    for cnt, col in enumerate(row):
        sheet_1.write(r, cnt, col)
        

##states sheet
c.execute('SELECT state_name, state_name, state_abbreviation FROM {tb}'.format(tb=tbl_focus_state))
all_rows = c.fetchall()

for r, row in enumerate(all_rows):
    column_state_abb = row[2]
    column_state_full = row[1]
    
    c.execute('SELECT t1.provider_id, t2.hospital_name, t2.city, t2.state, t2.county_name, t1.ranking FROM hospital_national_ranking as t1 JOIN hospital_general_information as t2 ON t1.provider_id = t2.provider_id WHERE t2.state="' + column_state_abb + '" ORDER BY CAST(t1.ranking AS INTEGER) ASC LIMIT 100')

    result_rows = c.fetchall()

    sheet_state_1 = workbook.add_worksheet(column_state_full)

    for r_temp, row_temp in enumerate(result_rows):
        for cnt_temp, col_temp in enumerate(row_temp):
            sheet_state_1.write(r_temp, cnt_temp, col_temp)
        
workbook.close()

print("Here we go," + file_name + " has been created/updated!")

###########################Create the Measures Statistical Analysis MS Excel Workbook###############################

file_name="measure_statistics.xlsx"
workbook = Workbook(file_name)

##Nationwide Sheet
sheet_1 = workbook.add_worksheet("Nationwide")

######Add to create time_effect_temp table


c.execute('DELETE FROM {tn} '.format(tn=tbl_time_effect_temp))
c.execute('INSERT INTO {tn_temp} SELECT t1.state, t1.measure_id, t1.measure_name, t1.score FROM {tn} as t1 WHERE printf("%d", t1.score) = t1.score'.\

        format(tn_temp= tbl_time_effect_temp, tn=tbl_time_effect))

c.execute('SELECT measure_id, min(score), max(score), stdev(score) FROM {tn_temp} GROUP BY measure_id ORDER BY measure_id'.format(tn_temp=tbl_time_effect_temp))

all_rows = c.fetchall()

for r, row in enumerate(all_rows):    
        for cnt, col in enumerate(row):
            sheet_1.write(r, cnt, col)

##states sheet
c.execute('SELECT state_name, state_name, state_abbreviation FROM {tb}'.format(tb=tbl_focus_state))
all_rows = c.fetchall()

for r, row in enumerate(all_rows):
    column_state_abb = row[2]
    column_state_full = row[1]
    
    c.execute('SELECT measure_id, min(score), max(score), stdev(score) FROM time_effect_temp WHERE state ="'+ column_state_abb +'" GROUP BY measure_id ORDER BY measure_id')

    result_rows = c.fetchall()

    sheet_state_1 = workbook.add_worksheet(column_state_full)

    for r_temp, row_temp in enumerate(result_rows):
        for cnt_temp, col_temp in enumerate(row_temp):
            sheet_state_1.write(r_temp, cnt_temp, col_temp)

workbook.close()

print("Here we go," + file_name + " has been created/updated!")

conn.close()


Here we go,hospital_ranking.xlsx has been created/updated!
Here we go,measure_statistics.xlsx has been created/updated!
